<a href="https://colab.research.google.com/github/nathsteve13/tugas-akhir/blob/main/src/ta_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import shutil, unicodedata
from PIL import Image, UnidentifiedImageError

In [3]:
DATA_DIR = "/content/drive/MyDrive/dataset_babybrandedshop"
import os
assert os.path.isdir(DATA_DIR), f"Folder dataset tidak ditemukan: {DATA_DIR}"
print("OK. Dataset folder ditemukan:", DATA_DIR)

OK. Dataset folder ditemukan: /content/drive/MyDrive/dataset_babybrandedshop


In [4]:
from pathlib import Path
import imghdr
import os


image_extensions = [".png", ".jpg", ".jpeg", ".gif", ".bmp", ".tif", ".tiff", ".webp"]
img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]

count_checked = 0
count_deleted = 0
count_error = 0

for filepath in Path(DATA_DIR).rglob("*"):
    if filepath.is_file() and filepath.suffix.lower() in image_extensions:
        count_checked += 1
        img_type = imghdr.what(filepath)

        if img_type is None or img_type not in img_type_accepted_by_tf:
            try:
                os.remove(filepath)
                print(f"[DELETED] {filepath} ({img_type})")
                count_deleted += 1
            except Exception as e:
                print(f"❌ Gagal menghapus {filepath}: {e}")
                count_error += 1

print("\n✅ Selesai.")
print(f"Total file dicek     : {count_checked}")
print(f"Total file dihapus   : {count_deleted}")
print(f"Total gagal dihapus  : {count_error}")


/tmp/ipython-input-1329616868.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr



✅ Selesai.
Total file dicek     : 3346
Total file dihapus   : 0
Total gagal dihapus  : 0


In [5]:
from pathlib import Path

TF_SUPPORTED = {".jpg", ".jpeg", ".png", ".gif", ".bmp"}
CONVERTIBLE  = {".webp", ".jfif", ".tif", ".tiff"}
data_path = Path(DATA_DIR)

class_dirs = [p for p in data_path.iterdir() if p.is_dir()]
class_names = sorted([p.name for p in class_dirs])

print(f"Jumlah kelas: {len(class_names)}")
print("Kelas:", class_names)

counts = {}
total = 0
for c in class_dirs:
    n = sum(1 for f in c.rglob("*") if f.suffix.lower() in TF_SUPPORTED)
    counts[c.name] = n
    total += n

print("\nRingkasan jumlah file per kelas:")
for k, v in counts.items():
    print(f"- {k}: {v}")
print(f"\nTotal gambar: {total}")

Jumlah kelas: 12
Kelas: ['.ipynb_checkpoints', 'backpack', 'bottlepacifier', 'boy16y', 'boy36m', 'girl16y', 'girl36m', 'hat', 'shoes', 'socks', 'stroller', 'swimsuit']

Ringkasan jumlah file per kelas:
- swimsuit: 311
- backpack: 303
- socks: 350
- hat: 300
- stroller: 298
- boy36m: 318
- girl36m: 270
- bottlepacifier: 299
- boy16y: 332
- girl16y: 250
- shoes: 315
- .ipynb_checkpoints: 0

Total gambar: 3346


In [6]:
IMG_SIZE   = (256, 256)
BATCH_SIZE = 32
VAL_SPLIT  = 0.2
SEED       = 42
AUTOTUNE   = tf.data.AUTOTUNE

In [7]:
preprocess = keras.Sequential([
    layers.Resizing(IMG_SIZE[0], IMG_SIZE[1]),
    layers.Rescaling(1./255),
], name="preprocess")

augment = keras.Sequential([
    layers.RandomFlip(mode="horizontal_and_vertical"),
    layers.RandomRotation(factor=0.0833, fill_mode="reflect"),
    layers.RandomZoom(height_factor=(-0.10, 0.10), width_factor=(-0.10, 0.10)),
    layers.RandomContrast(factor=0.2),
    layers.RandomBrightness(factor=0.1),
], name="augment")


In [8]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    str(DATA_DIR),
    labels="inferred",
    label_mode="categorical",
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

val_ds = keras.preprocessing.image_dataset_from_directory(
    str(DATA_DIR),
    labels="inferred",
    label_mode="categorical",
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)
class_names = train_ds.class_names
num_classes = len(class_names)
print("Kelas:", class_names, "| num_classes:", num_classes)

Found 3346 files belonging to 11 classes.
Using 2677 files for training.
Found 3346 files belonging to 11 classes.
Using 669 files for validation.
Kelas: ['backpack', 'bottlepacifier', 'boy16y', 'boy36m', 'girl16y', 'girl36m', 'hat', 'shoes', 'socks', 'stroller', 'swimsuit'] | num_classes: 11


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = (
    train_ds
    .map(lambda x, y: (augment(x, training=True), y), num_parallel_calls=AUTOTUNE)
    .map(lambda x, y: (preprocess(x), y), num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(1000)
    .prefetch(buffer_size=AUTOTUNE)
)

val_ds = (
    val_ds
    .map(lambda x, y: (preprocess(x), y), num_parallel_calls=AUTOTUNE)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for images, labels in train_ds.take(1):
    print("Bentuk gambar:", images.shape)
    print("Bentuk label:", labels.shape)

    plt.figure(figsize=(10, 10))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())
        plt.axis("off")

        class_index = np.argmax(labels[i].numpy())
        plt.title(class_names[class_index])
    plt.show()


In [ ]:
IMG_SHAPE = (256, 256, 3)
NUM_CLASSES = 11
LR = 1e-6

In [ ]:
# ====== 1) Encoder–Decoder (CAE) ======
def build_cae(img_shape=IMG_SHAPE):
    He = keras.initializers.HeNormal()

    def conv_bn_relu(x, filters, k=3):
        x = layers.Conv2D(filters, k, padding="same", use_bias=False, kernel_initializer=He)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        return x

    def deconv_bn_relu(x, filters, k=3, s=2):
        x = layers.Conv2DTranspose(filters, k, strides=s, padding="same", use_bias=False, kernel_initializer=He)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        return x

    inp = keras.Input(shape=img_shape)

    # ----- ENCODER: 32 -> 64 -> 128 -----
    x = conv_bn_relu(inp, 32)                 # 256x256x32
    x = layers.MaxPool2D(pool_size=2)(x)      # 128x128

    x = conv_bn_relu(x, 64)                   # 128x128x64
    x = layers.MaxPool2D(pool_size=2)(x)      # 64x64

    x = conv_bn_relu(x, 128)                  # 64x64x128
    bottleneck = layers.MaxPool2D(pool_size=2, name="bottleneck")(x)   # 32x32x128

    # ----- DECODER (simetris) -----
    y = deconv_bn_relu(bottleneck, 128, k=3, s=2)   # 64x64x128
    y = conv_bn_relu(y, 128)

    y = deconv_bn_relu(y, 64, k=3, s=2)             # 128x128x64
    y = conv_bn_relu(y, 64)

    y = deconv_bn_relu(y, 32, k=3, s=2)             # 256x256x32
    y = conv_bn_relu(y, 32)

    out = layers.Conv2D(3, 3, padding="same", activation="sigmoid", name="reconstruction")(y)

    cae = keras.Model(inp, out, name="CAE")
    encoder = keras.Model(inp, bottleneck, name="CAE_encoder")
    return cae, encoder

cae, encoder = build_cae()


In [ ]:
# ====== 2) Compile CAE (Eq. 2.13: MSE) ======
cae.compile(optimizer=keras.optimizers.Adam(LR),
            loss="mse",
            metrics=[keras.metrics.MeanAbsoluteError(name="mae")])

In [ ]:
# ====== 3) Siapkan dataset AE: (x, x) ======
def to_ae_pairs(x, y):
    return x, x

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
cae_train = train_ds.map(to_ae_pairs, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
cae_val   = val_ds.map(to_ae_pairs,   num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

In [ ]:
# ====== 4) Pre-train CAE ======
callbacks_ae = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=8, restore_best_weights=True, verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=4, min_lr=LR, verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="cae_best.weights.h5",
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    ),
    keras.callbacks.CSVLogger("cae_train_log.csv", append=False),
]
history_ae = cae.fit(
    cae_train,
    validation_data=cae_val,
    epochs=100,
    callbacks=callbacks_ae,
    verbose=1
)



In [ ]:
# ====== 5) CNN Classifier di atas BOTTLENECK ======
def build_classifier(encoder_model, num_classes=NUM_CLASSES, use_augment=True):
    encoder_model.trainable = False  # stage-1: freeze

    augment = keras.Sequential([
        layers.RandomFlip(mode="horizontal_and_vertical"),
        layers.RandomRotation(factor=0.0833, fill_mode="reflect"),
        layers.RandomZoom(height_factor=(-0.10, 0.10), width_factor=(-0.10, 0.10)),
        layers.RandomContrast(factor=0.2),
        layers.RandomBrightness(factor=0.1),
    ], name="augment") if use_augment else keras.Sequential(name="no_augment")

    He = keras.initializers.HeNormal()

    def conv_bn_relu(x, f, k=3):
        x = layers.Conv2D(f, k, padding="same", use_bias=False, kernel_initializer=He)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        return x

    inp = keras.Input(shape=IMG_SHAPE, name="clf_input")
    x = augment(inp)
    z = encoder_model(x)                       # 32x32x128 (dari CAE)

    x = conv_bn_relu(z, 128)
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dense(256, activation="relu", kernel_initializer=He)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(num_classes, activation="softmax", name="clf_output")(x)

    return keras.Model(inp, out, name="Hybrid_CAE_CNN")

clf = build_classifier(encoder)


In [ ]:
# ====== 6) Compile classifier (Eq. 2.14: Cross-Entropy) ======
clf.compile(optimizer=keras.optimizers.Adam(LR),
            loss="categorical_crossentropy",
            metrics=["accuracy"])

# Dataset untuk klasifikasi: gunakan (x, y) asli (bukan (x, x))
clf_train = train_ds.prefetch(AUTOTUNE)
clf_val   = val_ds.prefetch(AUTOTUNE)

In [ ]:
# ====== 8) Fine-tune tahap-2 (unfreeze sebagian encoder) ======
# Buka blok terakhir encoder agar representasi menyesuaikan task
for layer in encoder.layers[-4:]:   # sesuaikan kedalaman pembukaan
    layer.trainable = True

clf.compile(optimizer=keras.optimizers.Adam(LR),  # LR lebih kecil
            loss="categorical_crossentropy",
            metrics=["accuracy"])

callbacks_clf_stage2 = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=6, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=LR),
    keras.callbacks.ModelCheckpoint("hybrid_finetuned.keras", save_best_only=True, monitor="val_accuracy"),
]
history_clf_stage2 = clf.fit(clf_train, validation_data=clf_val, epochs=20, callbacks=callbacks_clf_stage2)

In [ ]:
final_metrics = clf.evaluate(clf_val, verbose=0)
print(dict(zip(clf.metrics_names, final_metrics)))

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Initialize lists to store true and predicted labels
y_true, y_pred = [], []

# Iterate over the validation dataset
for x_batch, y_batch in clf_val:
    # Get predictions for the current batch
    p = clf.predict(x_batch, verbose=0)

    # Convert one-hot encoded labels to class indices
    y_true.extend(np.argmax(y_batch.numpy(), axis=1))
    y_pred.extend(np.argmax(p, axis=1))

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate and print the confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

# Generate and print the classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

In [ ]:
import matplotlib.pyplot as plt

def plot_history(h, keys=("loss","val_loss")):
    plt.figure()
    for k in keys: plt.plot(h.history[k], label=k)
    plt.legend(); plt.xlabel("Epoch"); plt.grid(True)

# CAE (rekonstruksi)
plot_history(history_ae, ("loss","val_loss"))

# CNN stage-1 (head frozen)
plot_history(history_clf_stage1, ("loss","val_loss"))
plot_history(history_clf_stage1, ("accuracy","val_accuracy"))

# CNN stage-2 (unfreeze sebagian)
plot_history(history_clf_stage2, ("loss","val_loss"))
plot_history(history_clf_stage2, ("accuracy","val_accuracy"))
plt.show()


In [ ]:
import os

# Create the 'models' directory if it doesn't exist
os.makedirs("models", exist_ok=True)

# simpan model classifier final (struktur+weights)
clf.save("models/hybrid_classifier_savedmodel.keras")

# simpan encoder (untuk reuse / feature extractor)
encoder.save("models/encoder_savedmodel.keras")

# simpan CAE terbaik (optional untuk bab rekonstruksi)
# (jika kamu menyimpan checkpoint CAE sebelumnya)
# tf.keras.models.load_model("cae_best.keras").save("models/cae_savedmodel.keras")